<a href="https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_robust04_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyserini Demo on Robust04

This notebook provides a brief overview of how to use [Pyserini](https://github.com/castorini/anserini/blob/master/docs/pyserini.md), the Python interface to [Anserini](http://anserini.io), to search the collection from the TREC 2004 Robust Track.


## Installation


Install Python dependencies

In [0]:
%%capture
!pip install pyserini==0.8.1.0

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

Let's grab the pre-built index:

In [0]:
%%capture
!wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-robust04-20191213.tar.gz
# Alternate URL: https://www.dropbox.com/s/s91388puqbxh176/index-robust04-20191213.tar.gz
!tar xvfz index-robust04-20191213.tar.gz

Sanity check of index size:

In [3]:
!du -h index-robust04-20191213

2.1G	index-robust04-20191213


## SimpleSearcher Usage

You can use `pysearch` to search over an index. Here's the basic usage:

In [4]:
from pyserini.search import pysearch

searcher = pysearch.SimpleSearcher('index-robust04-20191213')
hits = searcher.search('black bear attacks')

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `hits` data structure holds the `docid`, the retrieval score, as well as the document content:

In [5]:
from IPython.core.display import display, HTML
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + hits[0].raw + '</div>'))

Here's how you can configure search options, such as BM25 parameters and using relevance feedback.

In [6]:
searcher.set_bm25_similarity(0.9, 0.4)
searcher.set_rm3_reranker(10, 10, 0.5)

hits2 = searcher.search('black bear attacks')

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


Note that the results have changed.

## IndexReaderUtils Usage

The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [7]:
from pyserini.index import pyutils

index_utils = pyutils.IndexReaderUtils('index-robust04-20191213/')

doc = index_utils.get_raw_document_contents('LA092790-0015')
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [8]:
analyzed = index_utils.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency):

In [9]:
doc_vector = index_utils.get_document_vector('LA092790-0015')
doc_vector

{'1990': 1,
 '20': 1,
 '27': 1,
 '45': 1,
 'about': 1,
 'abov': 1,
 'accustom': 1,
 'acr': 1,
 'advis': 2,
 'ago': 1,
 'all': 1,
 'although': 1,
 'amok': 1,
 'anim': 9,
 'anoth': 1,
 'area': 4,
 'arroyo': 1,
 'attack': 3,
 'author': 3,
 'awar': 1,
 'back': 1,
 'be': 1,
 'beach': 1,
 'bear': 4,
 'been': 11,
 'big': 1,
 'black': 2,
 'bobcat': 4,
 'brazen': 1,
 'california': 2,
 'call': 1,
 'capt': 1,
 'case': 1,
 'cat': 3,
 'citi': 1,
 'closer': 1,
 'come': 1,
 'common': 1,
 'commun': 1,
 'complaint': 1,
 'conflict': 1,
 'continu': 1,
 'control': 1,
 'counti': 4,
 'coyot': 10,
 'debusscher': 3,
 'deer': 3,
 'depart': 2,
 'develop': 1,
 "didn't": 1,
 'director': 1,
 'disappear': 1,
 'distanc': 1,
 'district': 1,
 'don': 1,
 'door': 1,
 'down': 1,
 'driest': 1,
 'drought': 4,
 'dry': 2,
 'east': 1,
 'eat': 1,
 'edg': 1,
 'edit': 1,
 'elsewher': 2,
 'especi': 2,
 'even': 1,
 'except': 1,
 'expect': 1,
 'fact': 1,
 'famili': 2,
 'fanci': 1,
 'far': 1,
 'feast': 1,
 'few': 2,
 'find': 1,
 'fi